In [50]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import yaml
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, WebsiteSearchTool
from dotenv import load_dotenv
import os


In [51]:


# Load environment variables
load_dotenv()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or ""
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY") or ""

In [52]:

# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']


In [53]:


# Initialize tools
serper_tool = SerperDevTool()
web_search_tool = WebsiteSearchTool()
# Tools not available in current CrewAI version
# serper_tool = None
# web_search_tool = None

# tools=[serper_tool, web_search_tool],
# guardrails=tasks_config['venue_booking_task']['guardrails'],
guardrails=tasks_config['venue_booking_task']['guardrails']

In [54]:


from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class EventPLan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

print("✅ Pydantic models created successfully!")


✅ Pydantic models created successfully!


In [55]:

# Creating Agents
venue_booking_agent = Agent(
    config=agents_config['venue_booking_agent']
)

logistics_agent = Agent(
    config=agents_config['logistics_agent']
)

marketing_agent = Agent(
    config=agents_config['marketing_agent']
)

# Creating Tasks
venue_booking_task = Task(
config=tasks_config['venue_booking_task'],
agent=venue_booking_agent,
tools=[serper_tool, web_search_tool],
guardrails=tasks_config['venue_booking_task']['guardrails']

      
)

logistics_task = Task(
config=tasks_config['logistics_task'],
agent=logistics_agent,
tools=[serper_tool, web_search_tool],
guardrails=tasks_config['logistics_task']['guardrails']  
)

marketing_task = Task(
config=tasks_config['marketing_task'],
agent=marketing_agent,
output_pydantic=EventPLan,
tools=[serper_tool, web_search_tool],
guardrails=tasks_config['marketing_task']['guardrails'] 

)



In [56]:

# Creating Crew
crew = Crew(
    agents=[
        venue_booking_agent,
        logistics_agent,
        marketing_agent
    ],
    tasks=[
        venue_booking_task,
        logistics_task,
        marketing_task
    ],
    verbose=True
)


In [57]:


# Event planning inputs
event_type = 'Corporate Conference'
industry = 'Technology'
event_objectives = 'Organize a successful technology conference for 200+ attendees'
event_team = """
- John Doe (Event Manager)
- Jane Doe (Venue Coordinator)
- Bob Smith (Marketing Specialist)
- Alice Johnson (Logistics Coordinator)
- Tom Brown (Technical Support)
"""
event_requirements = """
- Secure a venue with capacity for 200+ attendees and modern AV equipment
- Arrange catering services for coffee breaks and lunch
- Coordinate transportation and parking arrangements
- Create engaging marketing materials and social media campaigns
- Design professional invitations and registration system
- Ensure proper event insurance and security measures
- Coordinate with speakers and prepare presentation materials
- Set up event registration and check-in system
- Plan networking sessions and interactive activities
- Prepare post-event feedback collection and analysis
"""

print("✅ Event inputs defined successfully!")

# Prepare inputs for the crew
inputs = {
    'event_type': event_type,
    'event_objectives': event_objectives,
    'industry': industry,
    'event_team': event_team,
    'event_requirements': event_requirements
}

print("🚀 Starting the crew...")
result = crew.kickoff(inputs=inputs)
print("✅ Crew completed successfully!")

# Display the results
print("📊 Event Planning Results:")
print("=" * 50)
if hasattr(result, 'pydantic') and result.pydantic:
    result_dict = result.pydantic.dict()
    print("✅ Structured output received!")
    print(f"📋 Number of tasks: {len(result_dict.get('tasks', []))}")
    print(f"🎯 Number of milestones: {len(result_dict.get('milestones', []))}")
else:
    print("📝 Raw output:")
    print(result)

✅ Event inputs defined successfully!
🚀 Starting the crew...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 503d6071-9653-4301-893a-a0184c5ed504                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Venue Booking Specialist                                                                         │
│                                                                                                                 │
│  Task: Research and identify suitable venues for the Corporate Conference event, comparing options based on     │
│  capacity, location, availability, and budget constraints. Evaluate each venue for quality, accessibility, and  │
│  cost-effectiveness.                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Venue Booking Specialist                                                                         │
│                                                                                                                 │
│  Thought: I need to start by researching and comparing suitable venues for the Corporate Conference event       │
│  based on capacity, location, availability, and budget constraints. I will need to evaluate each venue for      │
│  quality, accessibility, and cost-effectiveness to provide a comprehensive final recommendation.                │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Event Marketing & Promotion Lead                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "tasks": [                                                                                                   │
│      {                                                                                                          │
│        "task_name": "Research best negotiation techniques with event service vendors",                          │
│        "estimated_time_hours": 2,                                                                               │
│        "required_resources": ["Search the internet with Serper"]                                                │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Explore effective social media marketing strategies for corporate events",                 │
│        "estimated_time_hours": 2,                                                                               │
│        "required_resources": ["Search the internet with Serper"]                                                │
│      }                                                                                                          │
│    ],                                                                                                           │
│    "milestones": [                                                                                              │
│      {                                                                                                          │
│        "milestone_name": "Vendor Negotiation Techniques Research",                                              │
│        "tasks": ["Research best negotiation techniques with event service vendors"]                             │
│      },                                                                                                         │
│      {                                                                                                          │
│        "milestone_name": "Social Media Marketing Strategy Exploration",                                         │
│        "tasks": ["Explore effective social media marketing strategies for corporate events"]                    │
│      }                                                                                                          │
│    ]                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a605d020-0ec3-4974-b033-d3b19c9367a8                                                                     │
│  Agent: Event Marketing & Promotion Lead                                                                        │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 503d6071-9653-4301-893a-a0184c5ed504                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "tasks": [                                                                                                   │
│      {                                                                                                          │
│        "task_name": "Research best negotiation techniques with event service vendors",                          │
│        "estimated_time_hours": 2,                                                                               │
│        "required_resources": ["Search the internet with Serper"]                                                │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Explore effective social media marketing strategies for corporate events",                 │
│        "estimated_time_hours": 2,                                                                               │
│        "required_resources": ["Search the internet with Serper"]                                                │
│      }                                                                                                          │
│    ],                                                                                                           │
│    "milestones": [                                                                                              │
│      {                                                                                                          │
│        "milestone_name": "Vendor Negotiation Techniques Research",                                              │
│        "tasks": ["Research best negotiation techniques with event service vendors"]                             │
│      },                                                                                                         │
│      {                                                                                                          │
│        "milestone_name": "Social Media Marketing Strategy Exploration",                                         │
│        "tasks": ["Explore effective social media marketing strategies for corporate events"]                    │
│      }                                                                                                          │
│    ]                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✅ Crew completed successfully!
📊 Event Planning Results:
✅ Structured output received!
📋 Number of tasks: 2
🎯 Number of milestones: 2
